## Regular Expressions

In this section we introduce regular expressions, an important tool to specify patterns in strings.

## Motivation

In a larger piece of text, many useful substrings come in a specific format. For instance, the sentence below contains a U.S. phone number.

`"give me a call, my number is 123-456-7890."`

The phone number contains the following pattern:

1. Three numbers
1. Followed by a dash
1. Followed by three numbers
1. Followed by a dash
1. Followed by four Numbers

Given a free-form segment of text, we might naturally wish to detect and extract the phone numbers. We may also wish to extract specific pieces of the phone numbers—for example, by extracting the area code we may deduce the locations of individuals mentioned in the text.

To detect whether a string contains a phone number, we may attempt to write a method like the following:

In [12]:
function is_phone_number(string)
    
    digits = "0123456789"
    
    function is_not_digit(token)
        return (!(token in digits))
    end
    
    # Three numbers
    for i in 1:3
        if is_not_digit(string[i])
            return false
        end
    end
    
    # Followed by a dash
    if string[4] != '-'
        return false
    end
    
    # Followed by three numbers
    for i in 5:7
        if is_not_digit(string[i])
            return false
        end
    end
        
    # Followed by a dash    
    if string[8] != '-'
        return false
    end
    
    # Followed by four numbers
    for i in 9:11
        if is_not_digit(string[i])
            return false
        end
    end
    
    return true
end

is_phone_number (generic function with 1 method)

In [13]:
is_phone_number("382-384-3840")

true

In [14]:
is_phone_number("phone number")

false

The code above is unpleasant and verbose. Rather than manually loop through the characters of the string, we would prefer to specify a pattern and command Julia to match the pattern.

**Regular expressions** (often abbreviated **regex**) conveniently solve this exact problem by allowing us to create general patterns for strings. Using a regular expression, we may re-implement the `is_phone_number` method in two short lines of Julia:

In [18]:
function is_phone_number(string)
    regex = r"[0-9]{3}-[0-9]{3}-[0-9]{4}"
    return typeof(match(regex, string)) != Nothing
end

is_phone_number("382-384-3840")

true

In the code above, we use the regex `[0-9]{3}-[0-9]{3}-[0-9]{4}` to match phone numbers. Although cryptic at a first glance, the syntax of regular expressions is fortunately much simpler to learn than the Julia language itself; we introduce nearly all of the syntax in this section alone. Check out the [docs](https://docs.julialang.org/en/v1/manual/strings/#Regular-Expressions-1) for more information on manipulating Strings with regular expressions.

## Regex Syntax

We start with the syntax of regular expressions. In Julia, regular expressions are most commonly stored as raw strings. Raw strings behave like normal Julia strings without special handling for backslashes.

For example, to store the string `hello \ world` in a normal Julia string, we must write:

In [20]:
some_string = "hello \\ world"
print(some_string)

hello \ world

Using a raw string removes the need to escape the backslash:

In [26]:
# Note the `r` prefix on the string
some_raw_string = raw"hello \ world"
print(some_raw_string)

hello \ world

Since backslashes appear often in regular expressions, we will use raw strings for all regexes in this section.

### Literals

A **literal** character in a regular expression matches the character itself. For example, the regex `r"a"` will match any `"a"` in `"Say! I like green eggs and ham!"`. All alphanumeric characters and most punctuation characters are regex literals.

In [66]:
# HIDDEN
function show_regex_match(text, regex)
    red = Base.text_colors[:red]
    default = Base.text_colors[:default]
    
    m = collect(eachmatch(regex, text))
    msg = text
    for x in m
        x_match = x.match
        msg2 = replace(msg, x_match => "$(red)$x_match$(default)")
        msg = msg2
    end

    println(msg)
end;

In [70]:
# The show_regex_match method highlights all regex matches in the input string
regex = r"green"
show_regex_match("Say! I like green eggs and ham!", regex)

Say! I like green eggs and ham!


In [71]:
show_regex_match("Say! I like green eggs and ham!", r"a")

Say! I like green eggs and ham!


In the example above we observe that regular expressions can match patterns that appear anywhere in the input string. In Julia, this behavior differs depending on the method used to match the regex—some methods only return a match if the regex appears at the start of the string; some methods return a match anywhere in the string.

Notice also that the `show_regex_match` method highlights all occurrences of the regex in the input string. Again, this differs depending on the Julia method used—some methods return all matches while some only return the first match.

Regular expressions are case-sensitive. In the example below, the regex only matches the lowercase `s` in `eggs`, not the uppercase `S` in `Say`.

In [72]:
show_regex_match("Say! I like green eggs and ham!", r"s")

Say! I like green eggs and ham!


### Wildcard Character

Some characters have special meaning in a regular expression. These meta characters allow regexes to match a variety of patterns.

In a regular expression, the period character `.` matches any character except a newline.

In [65]:
show_regex_match("The car parked in the garage.", r"[^c]ar")

The car parked in the garage.
